# Introduction

This notebook has 2 parts:
- Sourcing the DSM from the Vexcel API
- Converting that DSM into a LAS point cloud

It uses common data science and GIS python libraries:
- geopandas
- rasterio
- numpy
- PIL (Python Image Library)

It also uses laspy to handle the conversion of the numpy point cloud to LAS format.

## 1. Sourcing DSM data

There is only one input required, the coordinates of the bounding box of the polygon of interest. You could of course read in your own shapefile or load a WKT, but for simplicity we'll just use the bounding box here.

In [1]:
w, s, e, n = -117.9835, 34.1462, -117.9827, 34.1468

from shapely.geometry import box as geometry_from_bounding_box
import folium

aoi_geometry = geometry_from_bounding_box(w, s, e, n)

lon, lat = aoi_geometry.centroid.coords[0]
m = folium.Map(location=[lat, lon], tiles="cartodbpositron", zoom_start=20)

folium.GeoJson(
    aoi_geometry,
).add_to(m)
m

/Users/patrick/.pyenv/versions/3.7.3/lib/python3.7/site-packages/pandas/compat/__init__.py:84: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)
/Users/patrick/.pyenv/versions/3.7.3/lib/python3.7/site-packages/pandas/compat/__init__.py:84: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
import json
import math
import shutil
import sys

import requests
import rasterio
import numpy as np
import geopandas as gpd

from vdp_python_tools.coverage_api_utilities import create_coverage_dataframe
from vdp_python_tools.tile_math import num2deg, deg2num, degrees_per_pixel, pixel_to_coord, coord_to_pixel, tiles_in_polygon

Requests to our API are EPSG 4326 by default but can be adapted to other CRS if needed.

In [3]:
from vdp_python_tools.authentication import login
token = login()

In [4]:
import os
os.makedirs("./data/tmp", exist_ok=True)

In [5]:
aoi_geometry.exterior.coords[:]

[(-117.9827, 34.1462),
 (-117.9827, 34.1468),
 (-117.9835, 34.1468),
 (-117.9835, 34.1462),
 (-117.9827, 34.1462)]

**IMPORTANT**: zoom is set to 21 which is a very high resolution. For testing in larger areas, you might want to start higher e.g zoom = 16/17 to see how things line up and then run at 21 once you're happy.

In [6]:
#zoom = 16 # ~2.4m GSD
zoom = 21 # 0.075m GSD
tile_coords = tiles_in_polygon(aoi_geometry, 21, tile_footprint_filepath=f"./data/tmp/point_cloud_example_tile_prints.shp")

In [7]:
tile_coords

[(361272, 836713, 21),
 (361272, 836714, 21),
 (361272, 836715, 21),
 (361272, 836716, 21),
 (361273, 836713, 21),
 (361273, 836714, 21),
 (361273, 836715, 21),
 (361273, 836716, 21),
 (361274, 836713, 21),
 (361274, 836714, 21),
 (361274, 836715, 21),
 (361274, 836716, 21),
 (361275, 836713, 21),
 (361275, 836714, 21),
 (361275, 836715, 21),
 (361275, 836716, 21)]

In [8]:
print(f"Running {len(tile_coords)} requests to download {len(tile_coords)*256e3/1e9}Gb of data")

Running 16 requests to download 0.004096Gb of data


In [9]:
api = "GetDSMTile"

import os
os.makedirs(f"./data/tmp/point_cloud_testing", exist_ok=True)

for x, y, zoom in tile_coords:
    url = f"https://api.vexcelgroup.com/images/{api}/{zoom}/{x}/{y}?token={token}"# + "?" + "&".join([f"{key}={value}" for key, value in parameters.items()])
    response = requests.get(url, stream = True)
    if response.reason != "OK":
        raise Exception(f"API failed to get tile from api {api} at x={x} y={y} zoom={zoom} see url below \n {url}")
    filepath = f"./data/tmp/point_cloud_testing/dsm_img_{zoom}_{x}_{y}.tif"
    with open(filepath, 'wb') as out_file:
        shutil.copyfileobj(response.raw, out_file)


In [10]:
import os
folder_of_rasters = f"./data/tmp/point_cloud_testing"
rasters_to_merge = [f"./data/tmp/point_cloud_testing/{filename}" for filename in os.listdir(folder_of_rasters) if filename.endswith('.tif')]


In [11]:
rasters_to_merge = [f for f in rasters_to_merge if f"dsm_img_{zoom}_" in f] # because maybe you run 16 and then 20 - don't want to merge those

In [12]:
import rasterio

src_files_to_mosaic = []
for filepath in rasters_to_merge:
    src = rasterio.open(filepath)
    src_files_to_mosaic.append(src)

In [13]:
from rasterio.merge import merge


mosaic, out_trans = merge(src_files_to_mosaic)

In [14]:
from pyproj import CRS
crs = CRS.from_epsg(3857)
crs.to_string()

'EPSG:3857'

In [15]:
src.read_crs().to_string()

'EPSG:3857'

In [16]:
out_meta = src.meta.copy()

# Update the metadata
out_meta.update({"driver": "GTiff",
  "height": mosaic.shape[1],
  "width": mosaic.shape[2],
  "transform": out_trans,
  "crs": src.read_crs().to_string(), # Insert own CRS here
})

In [17]:
from rasterio.plot import show

filepath_mosaic = "./data/tmp/point_cloud_testing_mosaic.tif"

with rasterio.open(filepath_mosaic, "w", **out_meta) as dest:
    dest.write(mosaic)

In [18]:
from rasterio.transform import array_bounds

raster_array, raster_metadata = mosaic, out_meta

bounds = array_bounds(raster_metadata['height'], raster_metadata['width'], raster_metadata['transform'])

In [19]:
elevation_array = raster_array[0, :, :]
elevation_array = np.rot90(np.array(elevation_array), k=3)
pixels_wide, pixels_high = elevation_array.shape

deg_per_pixel_wide, deg_per_pixel_high = degrees_per_pixel(bounds, pixels_wide, pixels_high)

points_3d = []
for p_x in range(pixels_wide):
    for p_y in range(pixels_high):
        x, y = pixel_to_coord(p_x, p_y, bounds, deg_per_pixel_wide, deg_per_pixel_high)
        z = elevation_array[p_x, p_y]
        points_3d.append([x, y, z])

In [20]:
import laspy 
import pdal

def write_data(xyz, cloud_name, zoom):
    # 1. Create a new header
    header = laspy.LasHeader(point_format=0, version="1.2")
    #header.add_extra_dim(laspy.ExtraBytesParams(name="random", type=np.int32))
    #header.scales = np.array([0.1, 0.1, 0.1])

    header.offset = [0, 0, 0] # could need to make this ground level from DTM
    header.scales = np.array([1, 1, 0.01])

    # 2. Create a Las
    las = laspy.LasData(header)

    las.x = xyz[:, 0]
    las.y = xyz[:, 1]
    las.z = xyz[:, 2]
    #las.random = np.random.randint(-1503, 6546, len(las.points), np.int32)

    las.write(f"./data/tmp/point_cloud_{cloud_name}.las")
    
    pipeline_config = [
        {
            "filename": f"./data/tmp/point_cloud_{cloud_name}.las",
            "spatialreference":"EPSG:3857+3855"
        },
        {
            "type":"filters.reprojection",
            "in_srs":"EPSG:3857+3855",
            "out_srs":"EPSG:3857+3855"
        },
        {
            "filename": f"./data/tmp/point_cloud_{cloud_name}.las"
        }
    ]


    pipeline = pdal.Pipeline(json.dumps(pipeline_config))
    pipeline.execute()

ImportError: dlopen(/Users/patrick/.pyenv/versions/3.7.3/lib/python3.7/site-packages/pdal/libpdalpython.cpython-37m-darwin.so, 2): Library not loaded: @rpath/libpdalcpp.13.dylib
  Referenced from: /Users/patrick/.pyenv/versions/3.7.3/lib/python3.7/site-packages/pdal/libpdalpython.cpython-37m-darwin.so
  Reason: image not found

In [ ]:
write_data(np.array(points_3d), boundary_file_name, zoom)

In [21]:
!python --version

Python 3.7.3


In [26]:
!pip install PDAL

You should consider upgrading via the 'pip install --upgrade pip' command.


In [25]:
!pip freeze

affine==2.3.0
anyio==3.3.4
appdirs==1.4.4
appnope==0.1.2
argcomplete==1.12.3
argon2-cffi==21.1.0
astroid==2.4.2
attrs==19.3.0
Babel==2.9.1
backcall==0.2.0
black==19.10b0
bleach==4.1.0
boto3==1.9.179
botocore==1.12.244
bqplot==0.12.31
branca==0.4.2
certifi==2021.10.8
cffi==1.15.0
charset-normalizer==2.0.7
click==8.0.3
click-plugins==1.1.1
cligj==0.7.2
colour==0.1.5
cycler==0.11.0
debugpy==1.5.1
decorator==4.4.0
defusedxml==0.7.1
docutils==0.15.2
entrypoints==0.3
Fiona==1.8.20
Flask==2.0.2
Flask-Cors==3.0.10
folium==0.12.1
GDAL==3.4.0
geojson==2.5.0
geopandas==0.10.2
googledrivedownloader==0.4
greenlet==1.1.2
gunicorn==20.0.4
here-map-widget-for-jupyter==1.1.3
idna==3.3
imageio==2.13.1
importlib-metadata==4.8.1
importlib-resources==5.4.0
ipyevents==0.9.0
ipyfilechooser==0.6.0
ipykernel==6.5.0
ipyleaflet==0.14.0
ipython==7.29.0
ipython-genutils==0.2.0
ipytree==0.2.1
ipywidgets==7.6.5
isort==4.3.21
itsdangerous==2.0.1
jedi==0.18.0
Jinja2==3.0.2
jmespath==0.9.4
joblib==1.1.0
json5==0.9.6
js